# Single facet simulation

In [1]:
%matplotlib widget
import time
from bmcs_expsim.utils.mlab_decorators import decorate_figure
from mayavi import mlab
import numpy as np
np.seterr(divide='ignore', invalid='ignore') 
import warnings
import matplotlib.pylab as plt

In [2]:
from ibvpy.api import TStepBC
from ibvpy.bcond import BCSlice
from ibvpy.xmodel.xdomain_fe_grid import XDomainFEGrid
from ibvpy.xmodel.xdomain_interface import XDomainFEInterface

In [3]:
from ibvpy.fets import FETS2D4Q
from ibvpy.fets import FETS3D8H
from ibvpy.fets.fets1D5 import FETS1D52ULRH

In [4]:
from bmcs_matmod.slide.vslide_34 import Slide34
from ibvpy.tmodel.mats3D.mats3D_elastic.vmats3D_elastic import \
    MATS3DElastic
from ibvpy.tmodel.mats2D.mats2D_elastic.vmats2D_elastic import \
    MATS2DElastic
from bmcs_utils.api import Model

In [5]:
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField

In [6]:
n_e = 1
L_x = 1.0
xd_left = XDomainFEGrid(coord_min=(-1, 0),
                          coord_max=(0, 1),
                          shape=(1, 1),
                          fets=FETS2D4Q())
xd_right = XDomainFEGrid(coord_min=(0, 0),
                          coord_max=(1, 1),
                          shape=(1, 1),
                          fets=FETS2D4Q())

In [47]:
m1 = MATS2DElastic(E=28000, nu=0.3)
m2 = MATS2DElastic(E=28000, nu=0.3)

xd12 = XDomainFEInterface(
    I=xd_left.mesh.I[:, -1],
    J=xd_right.mesh.I[:, 0],
    fets=FETS1D52ULRH()
)

In [56]:
material_params = dict(
    E_T=1000, gamma_T=100, K_T=0, S_T=0.0001, c_T=1, bartau=3, 
    E_N=1000, S_N=0.0001, c_N = 1, m = 0.01, f_t=2, f_c=20, f_c0=10, eta=0.0 # 0.5
)
bond_m = Slide34(**material_params)

In [57]:
right_control = BCSlice(slice=xd_right.mesh[:, :, :, :],
                 var='u', dims=[0], value=0.02)
right_fixed = BCSlice(slice=xd_right.mesh[:, :, :, :],
                 var='u', dims=[1], value=0)
left_fixed = BCSlice(slice=xd_left.mesh[:, :, :, :],
                 var='u', dims=[0,1], value=0)
bc1 = [right_control, right_fixed, left_fixed]

In [58]:
m = TStepBC(
    domains=[
        (xd_left, m1),
        (xd_right, m2),
        (xd12, bond_m),
    ],
    bc=bc1,
)
m.hist.vis_record = {
    'strain': Vis3DTensorField(var='eps_ab'),
}
s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 0.1

In [59]:
s.run()

t:	 0.00(0), 
	 0.10(1), 
	 0.20(1), 
	 0.30(1), 
	 0.40(1), 
	 0.50(1), 
	 0.60(1), 
	 0.70(1), 
	 0.80(1), 
	 0.90(1), 
	 1.00(1), 



In [60]:
F_to = m.hist.F_t
U_to = m.hist.U_t
F_l_t = np.sum(F_to[:, right_control.dofs], axis=-1)
U_r_t = np.average(U_to[:, right_control.dofs], axis=-1)
_, ax = plt.subplots(1,1)
ax.plot(U_r_t, F_l_t)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
if False:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'strain'
    strain_viz = Viz3DTensorField(vis3d=m.hist['strain'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()

In [62]:
m.fe_domain[2].state_n

{'w_pi': array([[0.01799937, 0.01799937]]),
 's_pi_x': array([[0., 0.]]),
 's_pi_y': array([[0., 0.]]),
 'z': array([[-0.00133143, -0.00133143]]),
 'alpha_x': array([[0., 0.]]),
 'alpha_y': array([[0., 0.]]),
 'omega_T': array([[0., 0.]]),
 'omega_N': array([[0.24808186, 0.24808186]]),
 'sig_pi': array([[0., 0.]]),
 'tau_pi_x': array([[0., 0.]]),
 'tau_pi_y': array([[0., 0.]]),
 'Z': array([[0., 0.]]),
 'X_x': array([[0., 0.]]),
 'X_y': array([[0., 0.]]),
 'Y_T': array([[0., 0.]]),
 'Y_N': array([[0., 0.]])}